In [55]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from PIL import Image
from langchain_google_genai import GoogleGenerativeAIEmbeddings





load_dotenv()

class OpenAIModel:
    def __init__(self):
        api_key = os.environ["OPENAI_API_KEY"]
        if not api_key:
            raise ValueError("OPENAI_API_KEY is not set in the environment")
        self.model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.environ["OPENAI_API_KEY"])
    
    def complete(self, messages):
        return self.model.invoke(messages)


class GeminiModel:
    parser = StrOutputParser()
    def __init__(self, temperature = 0.7, max_output_tokens = None, top_p = None, top_k = None):
        api_key = os.environ["GEMINI_API_KEY"]
        if not api_key:
            raise ValueError("GEMINI_API_KEY is not set in the environment")
        self.model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key,
                                            temperature = temperature, max_output_tokens = max_output_tokens, top_p = top_p, top_k = top_k)
        self.vision_model = ChatGoogleGenerativeAI(model="gemini-pro-vision", google_api_key=api_key,
                                                   temperature = temperature, max_output_tokens = max_output_tokens, top_p = top_p, top_k = top_k)
        self.string_chain = self.model | self.parser
        self.embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001", 
            google_api_key=api_key
        )

    
    def _set_prompt(self, promptTemplate: ChatPromptTemplate):
        self.prompt_chain = promptTemplate | self.model | self.parser
    
    def complete(self, messages):
        return self.model.invoke(messages)
    
    def getTextResponse(self, messages):
        return self.string_chain.invoke(messages)
    
    def getEmbedding(self, text):
        return self.embeddings.embed_query("hello, world!")
    
    def getPromptResponse(self, data: dict):
        if not self.prompt_chain:
            raise ValueError("Prompt template is not set")
        return self.prompt_chain.invoke(data)
    
    def chatWithImage(self, imageData: str | Image.Image, prompt: str, messages = []):
        """Accepts image data with prompt for completion
        
            Parameters
            ----------
            imageData : str
                the data to image. Can be a URL or base64 encoded image or a PIL Image object or
                a filepath.
            prompt : str
                The text prompt to accompany the text
            messages : list
                The list of messages in the previous conversation
        """
        message = HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": prompt,
                },  # You can optionally provide text parts
                {"type": "image_url", "image_url": imageData},
            ]
        )
        return self.vision_model.invoke([*messages, message]).content

In [53]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(content="Translate the following from English into Italian: hi!"),
]

model = GeminiModel()

In [54]:
model.chatWithImage(
    imageData="videos/screenshot.jpg",
    prompt="Describe what's happening in this image.",
)

' A girl is sitting on a bed in her bedroom. She is looking out the window at the rain. The window is decorated with stars and flowers. The bed is covered in pillows and a blanket. There is a laptop on the bed. The girl is wearing a blue shirt and jeans. She has her hair in a ponytail. She is holding a cup of coffee.'

## Prompt tempaltes

In [39]:
# way 1

prompt_template = ChatPromptTemplate.from_messages(
    [("user", "Translate the following from English to {language}"), ("user", "{text}")]
)
result = prompt_template.invoke({
    "language": "Italian",
    "text": "hi!"
})
messages = result.to_messages()
model.getTextResponse(messages)

'ciao!'

In [56]:
# way 2

prompt_template = ChatPromptTemplate.from_messages(
    [("user", "Translate the following from English to {language}"), MessagesPlaceholder("messages")]
)
model._set_prompt(prompt_template)
model.getPromptResponse({
    "language": "Italian",
    "messages": [
        HumanMessage(content="hi!")
    ]
})

'ciao!'